# Assignment 1: Data Parsing, Cleansing and Integration
## Task 3
#### Student Name: Mrwan Alhandi
#### Student ID: s3969393

Date: Fri 8 Sep 2023

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used:
* pandas
* numpy
* from sklearn.model_selection import train_test_split
* from sklearn.linear_model import LinearRegression
* from sklearn.preprocessing import LabelEncoder
* from sklearn.impute import SimpleImputer
* from sklearn.metrics import mean_squared_error, r2_score


## Introduction
Welcome to this Jupyter Notebook, which focuses on a crucial step in the Data Science pipeline: Data Integration. The notebook aims to solve common challenges such as reconciling differing column names, handling extra columns, and standardizing different value representations for similar attributes. As data often comes from various SourceNames and in different formats, mastering these techniques is essential for any data science project. Follow along to learn how to create a cleaner, more unified dataset that is ready for analysis.

## 2.  Defining an Initial Global Schema

Let us define the Global Schema of the dataset required for pipeline:

- `SourceName` - A string indicate the SourceName of the job advertised.
- `Title` - A string indicate the title of the job advertised.
- `Location` - A string indicate the location of the job advertised.
- `Company` - A string indicate the company offering the job advertised.
- `ContractTime` - A string takes value from {'permanent','contract'} indicates whether the job is permanent or have contract time.
- `ContractType` - A string takes value from {'full_time','part_time'}.
- `Full-Time Equivalent (FTE)` - A float value from 0 - 1 used to describe the standard number of hours worked by a full-time employee in the organization. A value of 1 is often around 40 hours per week in many countries, although it can vary.
- `Category` - A string indicate the category of the job i.e Engineering, IT.. etc.
- `Salary` - A float indicate the yearly salary.
- `OpenDate` - Datetime data type indicate when the job offer is open and takes the format 'yyyy-mm-dd hh:mm:ss'.
- `CloseDate` - Datetime data type indicate when the job offer is close and takes the format 'yyyy-mm-dd hh:mm:ss'.

##  Import libraries 

In [1]:
import pandas as pd
import numpy as np
import regex as re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

### 1. Examining and loading data

In [2]:
# Read the dataset from task 1 and 2
df_1 = pd.read_csv('s3969393_dataset1_solution.csv')

df_1.head()

,id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,72218844,Tax Payroll Accountant,The City,Harrison Holgate,full_time,permanent,Accounting & Finance Jobs,54500.0,2013-11-13 00:00:00,2014-02-11 00:00:00,insurancejobs.co.uk
1,68621137,Commercial Account Executive/New Business Prod...,Ipswich,E Resources Ltd,full_time,permanent,Accounting & Finance Jobs,35000.0,2013-12-30 12:00:00,2014-03-30 12:00:00,insurancejobs.co.uk
2,70420238,Corporate Commercial Account Handler,Kent,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,28500.0,2012-05-09 12:00:00,2012-06-08 12:00:00,insurancejobs.co.uk
3,71391901,Senior Reinsurance Broker Back Up,The City,MW Appointments,full_time,permanent,Accounting & Finance Jobs,72500.0,2013-08-01 12:00:00,2013-09-30 12:00:00,insurancejobs.co.uk
4,70181615,Commercial Branch Director,The City,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,52500.0,2012-08-18 15:00:00,2012-10-17 15:00:00,insurancejobs.co.uk


In [3]:
# Read the data file
df_2 = pd.read_csv('s3969393_dataset2.csv')

df_2.head()

,Monthly Payment,Closing,Organisation,Location,Category,Full-Time Equivalent (FTE),Opening,Job Title,Type
0,1200.00,2012-12-27 12:00:00,Protocol Education,UK,Education,0.8,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),NaN
1,3125.00,2013-08-25 15:00:00,Cordius Ltd,UK,Information Technology,1.0,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent
2,4333.33,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,Information Technology,1.0,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent
3,3750.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales,1.0,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",NaN
4,2916.67,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales,1.0,2012-12-29 12:00:00,Channel Account Manager Software,NaN


In [4]:
# Display two dataframes together to compare
print('df_1:' + str(df_1.shape))
display(df_1.head())
print('df_2:' + str(df_2.shape))
display(df_2.head())

df_1:(50753, 11)


,id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,72218844,Tax Payroll Accountant,The City,Harrison Holgate,full_time,permanent,Accounting & Finance Jobs,54500.0,2013-11-13 00:00:00,2014-02-11 00:00:00,insurancejobs.co.uk
1,68621137,Commercial Account Executive/New Business Prod...,Ipswich,E Resources Ltd,full_time,permanent,Accounting & Finance Jobs,35000.0,2013-12-30 12:00:00,2014-03-30 12:00:00,insurancejobs.co.uk
2,70420238,Corporate Commercial Account Handler,Kent,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,28500.0,2012-05-09 12:00:00,2012-06-08 12:00:00,insurancejobs.co.uk
3,71391901,Senior Reinsurance Broker Back Up,The City,MW Appointments,full_time,permanent,Accounting & Finance Jobs,72500.0,2013-08-01 12:00:00,2013-09-30 12:00:00,insurancejobs.co.uk
4,70181615,Commercial Branch Director,The City,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,52500.0,2012-08-18 15:00:00,2012-10-17 15:00:00,insurancejobs.co.uk


df_2:(5000, 9)


,Monthly Payment,Closing,Organisation,Location,Category,Full-Time Equivalent (FTE),Opening,Job Title,Type
0,1200.00,2012-12-27 12:00:00,Protocol Education,UK,Education,0.8,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),NaN
1,3125.00,2013-08-25 15:00:00,Cordius Ltd,UK,Information Technology,1.0,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent
2,4333.33,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,Information Technology,1.0,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent
3,3750.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales,1.0,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",NaN
4,2916.67,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales,1.0,2012-12-29 12:00:00,Channel Account Manager Software,NaN


In [5]:
# Check the data types of each column
print(df_1.info())
print(df_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50753 entries, 0 to 50752
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            50753 non-null  int64  
 1   Title         50752 non-null  object 
 2   Location      50753 non-null  object 
 3   Company       50753 non-null  object 
 4   ContractType  50753 non-null  object 
 5   ContractTime  50753 non-null  object 
 6   Category      50753 non-null  object 
 7   Salary        50753 non-null  float64
 8   OpenDate      50752 non-null  object 
 9   CloseDate     50753 non-null  object 
 10  SourceName    50753 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 4.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Monthly Payment             5000 non-nul

#### Defining conflicts

### 2. Resolving schema conflicts
There are three conflicts that is found:
- Reconciling differing column names.
- Handling extra columns.
- Standardizing different value representations for similar attributes.

#### Conflict 1: df_1 SourceName & df_2 Full-Time Equivalent (FTE) attribute & ContractType
1. In the absence of explicit business requirements and comprehensive information regarding the dataset, it is challenging to ascertain the optimal treatment for these columns. The elimination of such columns would be a measure of last resort, as it could potentially result in the loss of critical data.

2. To address data integration issues, I have instituted a 'SourceName' attribute within the data structure. The notebooks from Task 1 and Task 2 will serve as reference points for populating any missing 'SourceName' fields during the dataframe merging process. Similarly to ContractType.

3. After thorough analysis, I have opted to generate a Full-Time Equivalent (FTE) attribute for the initial dataframe. A straightforward linear regression model will be employed to impute the missing FTE values, thereby enhancing the robustness and completeness of the dataset.


In [6]:
# Creating SourceName column for the second dataframe and filling it with missing values
df_2['SourceName'] = np.nan

In [7]:
# Creating ContractType
df_2['ContractType'] = np.nan

In [8]:
# Creating Full-Time Equivalent column for the first dataframe and filling it with missing values
df_1['Full-Time Equivalent (FTE)'] = np.nan

In [9]:
# Checking
df_1.head()

,id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName,Full-Time Equivalent (FTE)
0,72218844,Tax Payroll Accountant,The City,Harrison Holgate,full_time,permanent,Accounting & Finance Jobs,54500.0,2013-11-13 00:00:00,2014-02-11 00:00:00,insurancejobs.co.uk,NaN
1,68621137,Commercial Account Executive/New Business Prod...,Ipswich,E Resources Ltd,full_time,permanent,Accounting & Finance Jobs,35000.0,2013-12-30 12:00:00,2014-03-30 12:00:00,insurancejobs.co.uk,NaN
2,70420238,Corporate Commercial Account Handler,Kent,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,28500.0,2012-05-09 12:00:00,2012-06-08 12:00:00,insurancejobs.co.uk,NaN
3,71391901,Senior Reinsurance Broker Back Up,The City,MW Appointments,full_time,permanent,Accounting & Finance Jobs,72500.0,2013-08-01 12:00:00,2013-09-30 12:00:00,insurancejobs.co.uk,NaN
4,70181615,Commercial Branch Director,The City,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,52500.0,2012-08-18 15:00:00,2012-10-17 15:00:00,insurancejobs.co.uk,NaN


In [10]:
df_2.head()

,Monthly Payment,Closing,Organisation,Location,Category,Full-Time Equivalent (FTE),Opening,Job Title,Type,SourceName,ContractType
0,1200.00,2012-12-27 12:00:00,Protocol Education,UK,Education,0.8,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),NaN,NaN,NaN
1,3125.00,2013-08-25 15:00:00,Cordius Ltd,UK,Information Technology,1.0,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent,NaN,NaN
2,4333.33,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,Information Technology,1.0,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent,NaN,NaN
3,3750.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales,1.0,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",NaN,NaN,NaN
4,2916.67,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales,1.0,2012-12-29 12:00:00,Channel Account Manager Software,NaN,NaN,NaN


#### Conflict 2: df_1 Salary & df_2 Monthly Payement
The Global Schema defines Salary as a yearly integer type.

In [11]:
# We must multiply in Monthly Payement in df_2 by 12
df_2['Monthly Payment'] = df_2['Monthly Payment'] * 12
df_2.rename(columns={'Monthly Payment': 'Salary'}, inplace=True)


In [12]:
# let us change the data type of Salary in both dataframes to integer
df_1['Salary'] = df_1['Salary'].astype(float)
df_2['Salary'] = df_2['Salary'].astype(float)

In [13]:
df_1.head()

,id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName,Full-Time Equivalent (FTE)
0,72218844,Tax Payroll Accountant,The City,Harrison Holgate,full_time,permanent,Accounting & Finance Jobs,54500.0,2013-11-13 00:00:00,2014-02-11 00:00:00,insurancejobs.co.uk,NaN
1,68621137,Commercial Account Executive/New Business Prod...,Ipswich,E Resources Ltd,full_time,permanent,Accounting & Finance Jobs,35000.0,2013-12-30 12:00:00,2014-03-30 12:00:00,insurancejobs.co.uk,NaN
2,70420238,Corporate Commercial Account Handler,Kent,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,28500.0,2012-05-09 12:00:00,2012-06-08 12:00:00,insurancejobs.co.uk,NaN
3,71391901,Senior Reinsurance Broker Back Up,The City,MW Appointments,full_time,permanent,Accounting & Finance Jobs,72500.0,2013-08-01 12:00:00,2013-09-30 12:00:00,insurancejobs.co.uk,NaN
4,70181615,Commercial Branch Director,The City,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,52500.0,2012-08-18 15:00:00,2012-10-17 15:00:00,insurancejobs.co.uk,NaN


In [14]:
df_2.head()

,Salary,Closing,Organisation,Location,Category,Full-Time Equivalent (FTE),Opening,Job Title,Type,SourceName,ContractType
0,14400.00,2012-12-27 12:00:00,Protocol Education,UK,Education,0.8,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),NaN,NaN,NaN
1,37500.00,2013-08-25 15:00:00,Cordius Ltd,UK,Information Technology,1.0,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent,NaN,NaN
2,51999.96,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,Information Technology,1.0,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent,NaN,NaN
3,45000.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales,1.0,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",NaN,NaN,NaN
4,35000.04,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales,1.0,2012-12-29 12:00:00,Channel Account Manager Software,NaN,NaN,NaN


#### Conflict 3: df_2 Organisation, Type, Job Title, Closing, Opening
df_2 have different names to the same attributes defined in Global Schema.

In [15]:
df_2.rename(columns={'Organisation': 'Company'}, inplace=True)
df_2.rename(columns={'Type': 'ContractTime'}, inplace=True)
df_2.rename(columns={'Job Title': 'Title'}, inplace=True)
df_2.rename(columns={'Closing': 'CloseDate'}, inplace=True)
df_2.rename(columns={'Opening': 'OpenDate'}, inplace=True)

In [16]:
df_1.head()

,id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName,Full-Time Equivalent (FTE)
0,72218844,Tax Payroll Accountant,The City,Harrison Holgate,full_time,permanent,Accounting & Finance Jobs,54500.0,2013-11-13 00:00:00,2014-02-11 00:00:00,insurancejobs.co.uk,NaN
1,68621137,Commercial Account Executive/New Business Prod...,Ipswich,E Resources Ltd,full_time,permanent,Accounting & Finance Jobs,35000.0,2013-12-30 12:00:00,2014-03-30 12:00:00,insurancejobs.co.uk,NaN
2,70420238,Corporate Commercial Account Handler,Kent,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,28500.0,2012-05-09 12:00:00,2012-06-08 12:00:00,insurancejobs.co.uk,NaN
3,71391901,Senior Reinsurance Broker Back Up,The City,MW Appointments,full_time,permanent,Accounting & Finance Jobs,72500.0,2013-08-01 12:00:00,2013-09-30 12:00:00,insurancejobs.co.uk,NaN
4,70181615,Commercial Branch Director,The City,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,52500.0,2012-08-18 15:00:00,2012-10-17 15:00:00,insurancejobs.co.uk,NaN


In [17]:
df_2.head()

,Salary,CloseDate,Company,Location,Category,Full-Time Equivalent (FTE),OpenDate,Title,ContractTime,SourceName,ContractType
0,14400.00,2012-12-27 12:00:00,Protocol Education,UK,Education,0.8,2012-11-27 12:00:00,Higher Level Teaching Assistant (HLTA),NaN,NaN,NaN
1,37500.00,2013-08-25 15:00:00,Cordius Ltd,UK,Information Technology,1.0,2013-06-26 15:00:00,Software Developers / Software Architects Not...,Permanent,NaN,NaN
2,51999.96,2013-01-16 15:00:00,Understanding Recruitment,Walton-On-Thames,Information Technology,1.0,2012-10-18 15:00:00,Senior BI Consultant (Cognos),Permanent,NaN,NaN
3,45000.00,2013-02-13 15:00:00,Wallace Hind Selection,UK,Sales,1.0,2012-12-15 15:00:00,"Technical Sales Manager, Pigments Fillers to S...",NaN,NaN,NaN
4,35000.04,2013-01-12 12:00:00,Greenhill Group Ltd,Eastern England,Sales,1.0,2012-12-29 12:00:00,Channel Account Manager Software,NaN,NaN,NaN


### 3. Merging data

In [18]:
# Merge the two dataframes
df = pd.concat([df_1,df_2])
print (df.shape)
df.head(10)

(55753, 12)


,id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName,Full-Time Equivalent (FTE)
0,72218844.0,Tax Payroll Accountant,The City,Harrison Holgate,full_time,permanent,Accounting & Finance Jobs,54500.0,2013-11-13 00:00:00,2014-02-11 00:00:00,insurancejobs.co.uk,NaN
1,68621137.0,Commercial Account Executive/New Business Prod...,Ipswich,E Resources Ltd,full_time,permanent,Accounting & Finance Jobs,35000.0,2013-12-30 12:00:00,2014-03-30 12:00:00,insurancejobs.co.uk,NaN
2,70420238.0,Corporate Commercial Account Handler,Kent,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,28500.0,2012-05-09 12:00:00,2012-06-08 12:00:00,insurancejobs.co.uk,NaN
3,71391901.0,Senior Reinsurance Broker Back Up,The City,MW Appointments,full_time,permanent,Accounting & Finance Jobs,72500.0,2013-08-01 12:00:00,2013-09-30 12:00:00,insurancejobs.co.uk,NaN
4,70181615.0,Commercial Branch Director,The City,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,52500.0,2012-08-18 15:00:00,2012-10-17 15:00:00,insurancejobs.co.uk,NaN
5,70420144.0,Senior International Property Wordings Technician,The City,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,47500.0,2013-04-20 00:00:00,2013-07-19 00:00:00,insurancejobs.co.uk,NaN
6,67779521.0,Senior Commercial Business Development Manager,Lancashire,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,50000.0,2013-09-04 12:00:00,2013-12-03 12:00:00,insurancejobs.co.uk,NaN
7,70181605.0,Senior Marine Cargo Broker,The City,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,95000.0,2013-06-09 12:00:00,2013-07-09 12:00:00,insurancejobs.co.uk,NaN
8,72218956.0,Insurance Trainer,UK,Heat Recruitment,full_time,permanent,Accounting & Finance Jobs,30000.0,2013-06-01 12:00:00,2013-07-31 12:00:00,insurancejobs.co.uk,NaN
9,71138822.0,Professional Indemnity Development Underwriter,Hertfordshire,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,50000.0,2012-11-24 00:00:00,2013-02-22 00:00:00,insurancejobs.co.uk,NaN


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55753 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          50753 non-null  float64
 1   Title                       55752 non-null  object 
 2   Location                    55753 non-null  object 
 3   Company                     55267 non-null  object 
 4   ContractType                50753 non-null  object 
 5   ContractTime                54333 non-null  object 
 6   Category                    55753 non-null  object 
 7   Salary                      55753 non-null  float64
 8   OpenDate                    55752 non-null  object 
 9   CloseDate                   55753 non-null  object 
 10  SourceName                  50753 non-null  object 
 11  Full-Time Equivalent (FTE)  5000 non-null   float64
dtypes: float64(3), object(9)
memory usage: 5.5+ MB


### 4. Resolving data conflicts:
There are two conflict that was resolved. Duplications and data types.

#### Conflict 1: Duplications
Removing duplication instances after merging the two dataframes.

In [20]:
# Code to inspect data and identify conflicts in data
duplicates = df[df.duplicated(keep=False)] # showing all duplicated records
print ("There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=duplicates.columns.tolist()).head(10)

There are 0 duplicate records found


,id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName,Full-Time Equivalent (FTE)


In [21]:
df = df.drop_duplicates(keep='last')
df.shape

(55753, 12)

#### Finding global key for the data
[Company, Title]

In [22]:
# Code to identify global key
# check duplication
cols = ['Company','Title'] # this should uniquely identified a record, as the same bike could not be used by multiple trip started at the same time 
duplicates = df[df.duplicated(cols,keep=False)]
print ("There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=cols).head(10)

There are 296 duplicate records found


,id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName,Full-Time Equivalent (FTE)
46404,67957519.0,Construction Manager,London,1st Step Solutions Ltd,full_time,contract,Engineering Jobs,45600.00,2012-02-28 15:00:00,2012-03-13 15:00:00,justengineers.net,NaN
3661,NaN,Construction Manager,London,1st Step Solutions Ltd,NaN,Fixed Term Contract,Engineering,45600.00,2012-02-28 15:00:00,2012-03-13 15:00:00,NaN,1.0
4233,68626916.0,Market Risk Reporting/VBA Analyst,Central London,33 6 Consultancy Limited,full_time,permanent,IT Jobs,50000.00,2012-02-21 15:00:00,2012-04-21 15:00:00,jobserve.com,NaN
4177,NaN,Market Risk Reporting/VBA Analyst,Central London,33 6 Consultancy Limited,NaN,Permanent,Information Technology,50000.04,2012-02-21 15:00:00,2012-04-21 15:00:00,NaN,1.0
16096,68671836.0,Java eCommerce Consultant,Liverpool,365 Ltd,full_time,permanent,IT Jobs,50000.00,2013-06-01 00:00:00,2013-08-30 00:00:00,totaljobs.com,NaN
280,NaN,Java eCommerce Consultant,Liverpool,365 Ltd,NaN,Permanent,Information Technology,50000.04,2013-06-01 00:00:00,2013-08-30 00:00:00,NaN,1.0
19417,71306277.0,"Penetration Tester (CEH, Banking, Finance)",South East London,ANSON MCCADE,full_time,permanent,IT Jobs,60000.00,2012-04-05 12:00:00,2012-07-04 12:00:00,cwjobs.co.uk,NaN
3596,NaN,"Penetration Tester (CEH, Banking, Finance)",South East London,ANSON MCCADE,NaN,Permanent,Information Technology,60000.00,2012-04-05 12:00:00,2012-07-04 12:00:00,NaN,1.0
6582,67948667.0,Network Support Enginner,Central London,APM Resource Limited,full_time,permanent,IT Jobs,35000.00,2013-08-23 12:00:00,2013-09-22 12:00:00,jobsite.co.uk,NaN
1733,NaN,Network Support Enginner,Central London,APM Resource Limited,NaN,Permanent,Information Technology,35000.04,2013-08-23 12:00:00,2013-09-22 12:00:00,NaN,1.0


In [23]:
df = df.drop_duplicates(cols, keep='last')
df.shape

(55605, 12)

#### Conflict 2: creating id's for missing value
df_2 miss ids

In [24]:
# lets create a count ids for those rows that miss ids
# Generate unique IDs for instances that are missing this field
if 'id' in df.columns:
    max_id = df['id'].max()
else:
    max_id = 0

missing_id_indices = df[df['id'].isna()].index
for i, idx in enumerate(missing_id_indices):
    df.loc[idx, 'id'] = max_id + 1 + i

#### Conflict 3: Applying cleaning to merged dataframe

In [25]:
def clean_date(df):
    """
    Cleans the dataframe by addressing common errors and inconsistencies found in various attributes
    of this specific database schema. The function performs the following operations:

    1. Normalizes the 'SourceName' field to replace aliases with standard URLs or names.
    2. Cleans the 'Title' by removing extra spaces and certain special characters.
    3. Standardizes 'Location' names by replacing incorrect or alternative spellings.
    4. Strips extra spaces and replaces placeholder values in 'Company' with NaN.
    5. Cleans 'ContractTime' to replace placeholder values with NaN.
    6. Standardizes 'Salary' by converting all formats to a common scale.
    7. Converts 'OpenDate' and 'CloseDate' to pandas datetime format.
    
    Parameters:
    df (DataFrame): The DataFrame to be cleaned.

    Returns:
    DataFrame: The cleaned DataFrame.
    """
    
    # SourceName
    df['SourceName'] = df['SourceName'].replace('Jobcentre Plus', 'gov.uk/contact-jobcentre-plus')
    df['SourceName'] = df['SourceName'].replace('MyUkJobs', 'myukjob.com')
    df['SourceName'] = df['SourceName'].replace('GAAPweb', 'gaapweb.com')
    df['SourceName'] = df['SourceName'].replace('Brand Republic Jobs', 'onrec.com/directory/job-boards/brand-republic-jobs')
    df['SourceName'] = df['SourceName'].replace('eFinancialCareers', 'efinancialcareers.co.uk')
    df['SourceName'] = df['SourceName'].replace('PR Week Jobs', 'prweekjobs.co.uk')
    df['SourceName'] = df['SourceName'].replace('Multilingualvacancies', 'multilingualvacancies.com')
    df['SourceName'] = df['SourceName'].replace('Jobs Ac', 'jobs.ac.uk')
    df['SourceName'] = df['SourceName'].replace('Jobs24', 'jobs24.com')
    df['SourceName'] = df['SourceName'].replace('ijobs', 'ijobscenter.com')
    df['SourceName'] = df['SourceName'].replace('JobSearch', np.nan)
    df['SourceName'] = df['SourceName'].replace('JustLondonJobs', 'justlondonjobs.com')
    df['SourceName'] = df['SourceName'].replace('Teaching jobs - TES Connect', 'tes.com')
    df['SourceName'] = df['SourceName'].replace('TotallyExec', 'totallyexec.com')

    df['SourceName'] = df['SourceName'].str.strip()
    most_frequent_SourceName = df.groupby('Category')['SourceName'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    df['SourceName'].fillna(most_frequent_SourceName, inplace=True)

    # Title

    df['Title'] = df['Title'].str.replace(r'\s{2,}', ' ', regex=True)
    df['Title'] = df['Title'].str.replace(r'[*?]{1,}', '', regex=True)

    # Location

    df['Location'] = df['Location'].str.strip()
    df['Location'] = df['Location'].replace('Leads', 'Leeds')
    df['Location'] = df['Location'].replace('london', 'London')
    df['Location'] = df['Location'].replace('SURREY', 'Surrey')
    df['Location'] = df['Location'].replace('birmingham', 'Birmingham')
    df['Location'] = df['Location'].replace('Oxfords', 'Oxford')
    df['Location'] = df['Location'].replace('LANCASHIRE', 'Lancashire')
    df['Location'] = df['Location'].replace('HAMpshire', 'Hampshire')
    df['Location'] = df['Location'].replace('Londn', 'London')
    df['Location'] = df['Location'].replace('ABERDEEN', 'Aberdeen')
    df['Location'] = df['Location'].replace('DONCASTER', 'Doncaster')

    # Company
    df['Company'] = df['Company'].str.strip()
    df['Company'] = df['Company'].replace('N/A', np.nan)
    df['Company'] = df['Company'].replace('', np.nan)
    df['Company'] = df['Company'].replace('-', np.nan)

    most_frequent_company = df.groupby(['Location', 'Category'])['Company'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    df['Company'].fillna(most_frequent_company, inplace=True)

    most_frequent_company = df.groupby(['SourceName', 'Category'])['Company'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    df['Company'].fillna(most_frequent_company, inplace=True)

    # ContractTime
    df['ContractTime'] = df['ContractTime'].replace('N/A', np.nan)
    df['ContractTime'] = df['ContractTime'].replace('-', np.nan)
    df['ContractTime'] = df['ContractTime'].replace(' ', np.nan)

    most_frequent_contract_time = df.groupby('Company')['ContractTime'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    df['ContractTime'].fillna(most_frequent_contract_time, inplace=True)

    most_frequent_contract_time = df.groupby('Category')['ContractTime'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    df['ContractTime'].fillna(most_frequent_contract_time, inplace=True)

    # Salary
    mask = df['Salary'] == '-'
    df.loc[mask, 'Salary'] = np.NaN
    df['Salary'] = df['Salary'].astype(str)
    df['Salary'] = df['Salary'].str.replace(
    r'(\d+)\s*([-~]|to)\s*(\d+)', 
    lambda x: str((float(x.group(1)) + float(x.group(3))) / 2) if x.group(1) and x.group(3) else x.group(0),
    regex=True
    )
    df['Salary'] = df['Salary'].str.replace(r'[/]year|\s*per\s*?Annum', '', regex=True)
    df['Salary'] = df['Salary'].str.replace(r'(\d*.\d*)\sp[/]h', lambda x: str(float(x.group(1)) * 36 * 52), regex=True)
    df['Salary'] = df['Salary'].str.replace(r'(\d*.\d*)\sper\shour', lambda x: str(float(x.group(1)) * 36 * 52), regex=True)
    df['Salary'] = df['Salary'].str.replace(r'(\d*)k', lambda x: str(float(x.group(1)) * 1000), regex=True)
    mask = df['Salary'] == ' '
    df.loc[mask, 'Salary'] = np.NaN
    df["Salary"] = df["Salary"].astype("float")
    most_frequent_salary = df.groupby(['Company', 'Category'])['Salary'].transform(lambda x: x.mean() if not x.empty else np.nan)
    df['Salary'].fillna(most_frequent_salary, inplace=True)
    most_frequent_salary = df.groupby(['Company', 'ContractTime'])['Salary'].transform(lambda x: x.mean() if not x.empty else np.nan)
    df['Salary'].fillna(most_frequent_salary, inplace=True)
    most_frequent_salary = df.groupby(['Company', 'Location'])['Salary'].transform(lambda x: x.mean() if not x.empty else np.nan)
    df['Salary'].fillna(most_frequent_salary, inplace=True)
    most_frequent_salary = df.groupby(['Company'])['Salary'].transform(lambda x: x.mean() if not x.empty else np.nan)
    df['Salary'].fillna(most_frequent_salary, inplace=True)
    most_frequent_salary = df.groupby(['Category'])['Salary'].transform(lambda x: x.mean() if not x.empty else np.nan)
    df['Salary'].fillna(most_frequent_salary, inplace=True)
    df["Salary"] = df["Salary"].astype("float")

    # OpenDate
    df['OpenDate'] = pd.to_datetime(df['OpenDate'], errors='coerce')

    # CloseDate
    df['CloseDate'] = pd.to_datetime(df['CloseDate'], errors='coerce')


    return df

df = clean_date(df)


In [26]:
df.head(10)

,id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName,Full-Time Equivalent (FTE)
0,72705204.0,Tax Payroll Accountant,The City,Harrison Holgate,full_time,permanent,Accounting & Finance Jobs,54500.0,2013-11-13 00:00:00,2014-02-11 00:00:00,insurancejobs.co.uk,NaN
1,72705205.0,Commercial Account Executive/New Business Prod...,Ipswich,E Resources Ltd,full_time,permanent,Accounting & Finance Jobs,35000.0,2013-12-30 12:00:00,2014-03-30 12:00:00,insurancejobs.co.uk,NaN
2,72705206.0,Corporate Commercial Account Handler,Kent,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,28500.0,2012-05-09 12:00:00,2012-06-08 12:00:00,insurancejobs.co.uk,NaN
3,72705207.0,Senior Reinsurance Broker Back Up,The City,MW Appointments,full_time,permanent,Accounting & Finance Jobs,72500.0,2013-08-01 12:00:00,2013-09-30 12:00:00,insurancejobs.co.uk,NaN
4,72705208.0,Commercial Branch Director,The City,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,52500.0,2012-08-18 15:00:00,2012-10-17 15:00:00,insurancejobs.co.uk,NaN
5,72705209.0,Senior International Property Wordings Technician,The City,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,47500.0,2013-04-20 00:00:00,2013-07-19 00:00:00,insurancejobs.co.uk,NaN
6,72705210.0,Senior Commercial Business Development Manager,Lancashire,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,50000.0,2013-09-04 12:00:00,2013-12-03 12:00:00,insurancejobs.co.uk,NaN
7,72705211.0,Senior Marine Cargo Broker,The City,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,95000.0,2013-06-09 12:00:00,2013-07-09 12:00:00,insurancejobs.co.uk,NaN
8,72705212.0,Insurance Trainer,UK,Heat Recruitment,full_time,permanent,Accounting & Finance Jobs,30000.0,2013-06-01 12:00:00,2013-07-31 12:00:00,insurancejobs.co.uk,NaN
9,72705213.0,Professional Indemnity Development Underwriter,Hertfordshire,Lawes Insurance Recruitment,full_time,permanent,Accounting & Finance Jobs,50000.0,2012-11-24 00:00:00,2013-02-22 00:00:00,insurancejobs.co.uk,NaN


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55605 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id                          55605 non-null  float64       
 1   Title                       55604 non-null  object        
 2   Location                    55605 non-null  object        
 3   Company                     55605 non-null  object        
 4   ContractType                50605 non-null  object        
 5   ContractTime                55605 non-null  object        
 6   Category                    55605 non-null  object        
 7   Salary                      55605 non-null  float64       
 8   OpenDate                    55604 non-null  datetime64[ns]
 9   CloseDate                   55605 non-null  datetime64[ns]
 10  SourceName                  50605 non-null  object        
 11  Full-Time Equivalent (FTE)  5000 non-null   float64    

#### Conflict 4: Applying linear regression model to predict Full-Time Equivalent (FTE)

In [28]:
df['Full-Time Equivalent (FTE)'].value_counts()

1.0    4834
0.4      47
0.8      41
0.6      39
0.2      39
Name: Full-Time Equivalent (FTE), dtype: int64

In [29]:
# Handle missing values for training
imputer_mode = SimpleImputer(strategy='most_frequent')
df['ContractType'] = imputer_mode.fit_transform(df[['ContractType']])
df['ContractTime'] = imputer_mode.fit_transform(df[['ContractTime']])

# Drop rows where 'Full-Time Equivalent (FTE)' is NaN, as these cannot be used for training
train_df = df.dropna(subset=['Full-Time Equivalent (FTE)'])

# Encode Categorical Variables
label_encoder = LabelEncoder()
train_df['ContractType_encoded'] = label_encoder.fit_transform(train_df['ContractType'])
train_df['ContractTime_encoded'] = label_encoder.fit_transform(train_df['ContractTime'])

# Prepare X and Y Data
X = train_df[['ContractType_encoded', 'ContractTime_encoded']]
y = train_df['Full-Time Equivalent (FTE)']

# Split Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and Fit Linear Regression Model
model = LinearRegression()
model.fit(X_train, y_train)

# Make Predictions on Test Set
y_pred = model.predict(X_test)

# Evaluate Model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')

# Update encoded columns in the original DataFrame
df['ContractType_encoded'] = label_encoder.fit_transform(df['ContractType'])
df['ContractTime_encoded'] = label_encoder.fit_transform(df['ContractTime'])

# Find indices of missing FTE values
missing_fte_indices = df[df['Full-Time Equivalent (FTE)'].isna()].index

# Predict and update only the missing FTE values
df.loc[missing_fte_indices, 'Full-Time Equivalent (FTE)'] = model.predict(df.loc[missing_fte_indices, ['ContractType_encoded', 'ContractTime_encoded']])



Mean Squared Error: 0.007261890608837024
R2 Score: 0.03249096576002619


/var/folders/b4/8qmz0c3s3qq6pc__ps0x3pzh0000gn/T/ipykernel_5755/1945151356.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['ContractType_encoded'] = label_encoder.fit_transform(train_df['ContractType'])
/var/folders/b4/8qmz0c3s3qq6pc__ps0x3pzh0000gn/T/ipykernel_5755/1945151356.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['ContractTime_encoded'] = label_encoder.fit_transform(train_df['ContractTime'])


In [30]:
# let us round to the nearest integer
df['Full-Time Equivalent (FTE)'] = df['Full-Time Equivalent (FTE)'].round(0)


In [31]:
df['Full-Time Equivalent (FTE)'].value_counts()

1.0    55604
0.0        1
Name: Full-Time Equivalent (FTE), dtype: int64

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55605 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id                          55605 non-null  float64       
 1   Title                       55604 non-null  object        
 2   Location                    55605 non-null  object        
 3   Company                     55605 non-null  object        
 4   ContractType                55605 non-null  object        
 5   ContractTime                55605 non-null  object        
 6   Category                    55605 non-null  object        
 7   Salary                      55605 non-null  float64       
 8   OpenDate                    55604 non-null  datetime64[ns]
 9   CloseDate                   55605 non-null  datetime64[ns]
 10  SourceName                  50605 non-null  object        
 11  Full-Time Equivalent (FTE)  55605 non-null  float64    

In [33]:
# we dont need the encoded columns anymore
df.drop(['ContractType_encoded', 'ContractTime_encoded'], axis=1, inplace=True)

#### Conflict 5: data types
Converting columns to the appropriate data types.

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55605 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id                          55605 non-null  float64       
 1   Title                       55604 non-null  object        
 2   Location                    55605 non-null  object        
 3   Company                     55605 non-null  object        
 4   ContractType                55605 non-null  object        
 5   ContractTime                55605 non-null  object        
 6   Category                    55605 non-null  object        
 7   Salary                      55605 non-null  float64       
 8   OpenDate                    55604 non-null  datetime64[ns]
 9   CloseDate                   55605 non-null  datetime64[ns]
 10  SourceName                  50605 non-null  object        
 11  Full-Time Equivalent (FTE)  55605 non-null  float64    

In [35]:
# id should be integer
df['id'] = df['id'].astype(int)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55605 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id                          55605 non-null  int64         
 1   Title                       55604 non-null  object        
 2   Location                    55605 non-null  object        
 3   Company                     55605 non-null  object        
 4   ContractType                55605 non-null  object        
 5   ContractTime                55605 non-null  object        
 6   Category                    55605 non-null  object        
 7   Salary                      55605 non-null  float64       
 8   OpenDate                    55604 non-null  datetime64[ns]
 9   CloseDate                   55605 non-null  datetime64[ns]
 10  SourceName                  50605 non-null  object        
 11  Full-Time Equivalent (FTE)  55605 non-null  float64    

### 5. Saving the integrated and reshaped data

In [37]:
# code to save output data
df.to_csv('s3969393_dataset_integrated.csv', index=False)

## Summary of the Assessment Task
This notebook addresses the pivotal step of Data Integration in the field of Data Science. It tackles common challenges such as harmonizing divergent column names, managing extraneous columns, and unifying disparate value representations under a common standard. By adhering to a global data schema defined at the onset, the notebook successfully combines two distinct dataframes into a unified, cohesive dataset ready for further analysis.